# URL score calculator to determine if it is malicious or not.

## Models make use of Word2Vec (CBOW) and FastText for certain components

## imports

In [1]:
import pandas as pd
import keras
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model
from urllib.parse import urlparse,urlsplit
import gensim
from gensim.models import Word2Vec, FastText  

Using TensorFlow backend.
C:\Users\adessai\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\adessai\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\adessai\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\adessai\anaconda3\lib\site-packages\tensorflow\python\framework\d

## Prepare input data :

## split

In [2]:
def splitter(url_list):
    new_df=pd.DataFrame(columns=["scheme","netloc","path","query","fragment"])
    for url in url_list:
    #     test = [urlsplit(url).scheme, urlsplit(url).netloc, urlsplit(url).path, urlsplit(url).query, urlsplit(url).fragment]
        split_result=urlsplit(url)
        scheme = split_result.scheme
        netloc = split_result.netloc
        path = split_result.path
        query = split_result.query
        new_df =  new_df.append({'scheme': scheme, 'netloc': netloc, 'path': path, 'query': query}, ignore_index= True)
        new_df.fillna(0,inplace=True)
    return (new_df)

## vectorise using word2vec (cbow)

In [3]:
def vectorizer(new_df):
    
    model1 = Word2Vec.load("w2v_url")
    model2 = Word2Vec.load("w2v_path")
    model3 = Word2Vec.load("w2v_query")
    
    
    splitr1= new_df["netloc"]
    splitr2= new_df["path"]
    splitr3= new_df["query"]

    splitr1_1= [str(val).split(".") for val in splitr1.tolist()]
    splitr2_1= [str(val).split("/") for val in splitr2.tolist()]
    splitr3_1= [str(val).split("=") for val in splitr3.tolist()]
    print(len(splitr1_1),len(splitr2_1),len(splitr3_1))
    
    model1.build_vocab(splitr1_1, update=True)
    model1.train(splitr1_1, total_examples=len(splitr1_1), epochs=1)

    model2.build_vocab(splitr2_1, update=True)
    model2.train(splitr2_1, total_examples=len(splitr1_1), epochs=1)

    model3.build_vocab(splitr3_1, update=True)
    model3.train(splitr3_1, total_examples=len(splitr1_1), epochs=1)
    
    holder1=[]
    vec_df1= pd.DataFrame()
    val=[0 for i in range(100)]
    for i in range(len(splitr1_1)):
        for j in range(len(splitr1_1[i])):
            val += model1[splitr1_1[i][j]]
        holder1.append(val.tolist())
        val *=0
    vec_df1=vec_df1.append(holder1)   
    vec_df1.head()

    holder2=[]
    vec_df2= pd.DataFrame()
    val=[0 for i in range(100)]
    for i in range(len(splitr2_1)):
        for j in range(len(splitr2_1[i])):
            val += model2[splitr2_1[i][j]]
        holder2.append(val.tolist())
        val *=0
    vec_df2=vec_df2.append(holder2) 
    vec_df2.head()

    holder3=[]
    vec_df3= pd.DataFrame()
    val=[0 for i in range(100)]
    for i in range(len(splitr3_1)):
        for j in range(len(splitr3_1[i])):
            val += model3[splitr3_1[i][j]]
        holder3.append(val.tolist())
        val *=0
    vec_df3=vec_df3.append(holder3) 
    vec_df3.head()
    
    
    tester_df_batch=pd.DataFrame()
    tester_df_batch=pd.concat([vec_df1,vec_df2,vec_df3], axis=1)
    
    return(tester_df_batch)

# URL score prediction, update, endpoint mischief scorer :

## url score updater

In [4]:
def return_avg_for_updated_url_score (eps,model):
    switch = []
    for key in eps.keys():
        for val in range(len(eps[key])):
            switch+=[(eps[key][val],key,calc_endpoint_score(model.predict(vectorizer(splitter(eps[key]))).tolist()))]

    y =pd.DataFrame(switch, columns=['url','ip','eps_score'])

    grouped_sum = y.groupby('url').sum()
    grouped_size = y.groupby('url').size()
    # grouped_size['a']


    unique_urls = list(y['url'].unique())
    avg_scores_of_endpoints_hitting_the_same_url =[]
    for url in unique_urls :
        avg_scores_of_endpoints_hitting_the_same_url += [{url:grouped_sum['eps_score'][url]/grouped_size[url]}]

    return(avg_scores_of_endpoints_hitting_the_same_url,unique_urls)    

## endpoint score calculator

In [5]:
def calc_endpoint_score(score):
    day_score = float()
    bad_counter = int()
    for sc in score:

        if sc[0] > 0.5:
            day_score += sc[0]
            bad_counter += 1
        else:
            day_score += sc[0]

    bad_counter
    ep_w = (day_score/len(score))*(1.0-(1-(bad_counter/len(score))))
    return(ep_w)

## url name_vector relatedness 

In [6]:
def vector_relatedness(url):
    w_ = urlsplit(url)
    w_netloc = w_.netloc.split('.')
    w_path = w_.path.split('.')
    
    if(w_netloc[0]==''):

        related_word=w_path
        if(related_word[0] == 'www'):
            related_word_finder = related_word[1]

        else:
            related_word_finder = related_word[0]

    elif(w_netloc[0]=='www'):
        related_word_finder = w_netloc[1]

    else:
        related_word_finder = w_netloc[0]
        
        
    FastText_model = FastText.load('ft_.model')
    relatedness_vector = FastText_model.wv.most_similar_cosmul(related_word_finder)
    rv =np.mean([i[1] for i in relatedness_vector])
    return(rv)

## updating url scores based on relatedness and endpoint scores 

In [7]:
def url_score_recalculator(val_avg, unique_urls, url_scores):
    val_avg = val_avg
    unique_urls = unique_urls
    url_scores = [url_scores[i][0] for i in range(len(url_scores))]
    avg_list = [float(str(*list(val_avg[i].values()))) for i in range(len(val_avg))]
    
    rv =[vector_relatedness(url) for url in unique_urls]
    m_rv_url_score = np.add(np.multiply(url_scores,0.85),np.multiply(rv,0.1))
    
    m_avg_list = np.multiply(avg_list,0.05)
    
    updated_scores = np.subtract(m_rv_url_score,m_avg_list)
    updated_url_score_dict = [{unique_urls[i]:updated_scores[i]} for i in range(len(unique_urls))]
    return(updated_url_score_dict)

## inputs and outputs

In [8]:
eps = {'192.168.6.6.1':["http://zastapiony.piklamp.nl/military.xbel?face=jlthEvD&oil=70UfcEdppE&similar=0T9GXVd&plane=&never=XzUI&size=h-S&building=3NHL3LH4j&play=Ke9C4&over=&pattern=gKK","https://www.google.com"],
      '0.0.0.1':["https://www.yagle.com/abcd/?=123","https://www.google.com","www.bitsadmin.com/"]}

### on assuming an input of the above format to the code

model = load_model('current.best__std.hdf5')
val_avg, unique_urls = return_avg_for_updated_url_score(eps,model)

final_vec = vectorizer(splitter(eps['0.0.0.1']))
url_scores = model.predict(vectorizer(splitter(unique_urls))).tolist()
updated_url_score_dict = url_score_recalculator(val_avg, unique_urls, url_scores)

score =  model.predict(final_vec).tolist()
ep_sc = calc_endpoint_score(score)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\adessai\anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2 2 2


C:\Users\adessai\anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\adessai\anaconda3\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\adessai\anaconda3\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


2 2 2
3 3 3
3 3 3
3 3 3
3 3 3
4 4 4


In [16]:
print("Endpoint Vectors for URLs of '0.0.0.1' (3 x 300) : ",final_vec)
print("\nScores for all unique URLs not bound by the endpoint: \n",url_scores)
print("\nUpdated URL scores to dict: \n",updated_url_score_dict)
print("\nScores for URLs of '0.0.0.1' endpoint: \n",score)
print("\nEndpoint score: ",ep_sc)

Endpoint Vectors for URLs of '0.0.0.1' (3 x 300) :           0         1         2         3         4         5         6   \
0  1.459082 -3.259299  1.979346 -0.265111 -2.896737  1.215932 -1.661186   
1  3.578903 -5.270251  2.364378  0.229436 -3.393633  1.455947 -2.137968   
2 -0.145892 -0.154081 -0.383800 -0.552012 -1.168146 -0.070009  0.452086   

         7         8         9   ...        90        91        92        93  \
0  0.168161 -1.645257 -1.127592  ...  0.199735 -0.440787 -0.280912  0.482175   
1  0.025913 -2.056639 -2.126576  ...  0.211939 -0.424193 -0.261291  0.470320   
2  0.525980  1.194449 -1.202422  ...  0.211939 -0.424193 -0.261291  0.470320   

         94        95        96        97        98        99  
0 -1.150446  0.503363 -0.022905 -0.048369  0.052436  1.149770  
1 -1.098585  0.471811  0.010624 -0.075901  0.035317  1.103188  
2 -1.098585  0.471811  0.010624 -0.075901  0.035317  1.103188  

[3 rows x 300 columns]

Scores for all unique URLs not bound by the e

## future updates would make use of incremental learning structures